# Rag Simple Financial Analisys (fundamental Analysis)

### Fundamental vs. Technical Analysis: An Overview
There are two main schools of thought when evaluating investments and making trading decisions: fundamental analysis and technical analysis. While both aim to predict future price movements and identify profitable opportunities, they take very different approaches. Fundamental analysis examines an asset's intrinsic value by examining economic and financial factors, while technical analysis focuses solely on price action and chart patterns. 


### Company's balance sheet
A company's balance sheet, also known as a "statement of financial position," reveals the firm's assets, liabilities, and owners' equity (net worth) at a specific point in time. The balance sheet, together with the income statement and cash flow statement, make up the cornerstone of any company's financial statements.

### What Is an Income Statement?
An income statement is a financial report used by a business. It tracks the company's revenue, expenses, gains, and losses during a set period. Also known as the profit and loss (P&L) statement or the statement of revenue and expense, it provides valuable insights into a company’s operations, the efficiency of its management, underperforming sectors, and its performance relative to industry peers.
### Cash Flow Statement

A cash flow statement is a financial statement that provides aggregate data regarding all cash inflows that a company receives from its ongoing operations and external investment sources. It also includes all cash outflows that pay for business activities and investments during a given period. 


#### LINKS
- https://www.investopedia.com/ask/answers/difference-between-fundamental-and-technical-analysis/
- https://uk.finance.yahoo.com/
- https://ranaroussi.github.io/yfinance/reference/index.html
- https://site.financialmodelingprep.com/
- https://pypi.org/project/fmpsdk/
- https://www.morningstar.com/business/brands/data-analytics/products/direct/features/python-package
- https://www.quantconnect.com/docs/v2/cloud-platform/api-reference


In [ ]:

import time
import pandas as pd

import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker

#streamlit
import streamlit as st

#rag library
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document

from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.storage import InMemoryStore

import pandas as pd
import json

from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [ ]:
from dotenv import load_dotenv, dotenv_values
import os
import pandas as pd

env_path = os.path.join( ".env")
load_dotenv(env_path)

In [ ]:
ROOT_DIR = os.getcwd()

with open(os.path.join(ROOT_DIR, "complete-tube-421007-208a4862c992.json",)) as source:
    info = json.load(source)

In [ ]:
from google.oauth2 import service_account
import vertexai

In [ ]:
GEMINI_API_KEY = os.getenv("GEMINI-API-KEY")
vertex_credentials = service_account.Credentials.from_service_account_info(
                info
            )

In [ ]:
key = os.getenv("GEMINI_API_KEY") #API KEY
embeddings = GoogleGenerativeAIEmbeddings(google_api_key=key, model="models/embedding-001",credentials=vertex_credentials)
#define LLM
llm = ChatGoogleGenerativeAI(google_api_key=key, temperature=0.01, model="gemini-1.5-pro",credentials=vertex_credentials)

In [ ]:
def table_transform(df):
  table = df
  table = table.transpose()
  col = table.iloc[0].str.strip()
  table = table[1:]
  table.columns = col
  table.replace('-- ', 0, inplace=True)
  table = table.replace(',', '', regex=True).astype(float)
  table = table.iloc[::-1]

  return table

def format_numbers(x, pos):
    sign = '-' if x < 0 else ''
    x = abs(x)

    if x >= 1e12:
        return f'{sign}{x*1e-12:.1f}T'
    elif x >= 1e9:
        return f'{sign}{x*1e-9:.1f}B'
    elif x >= 1e6:
        return f'{sign}{x*1e-6:.1f}M'
    else:
        return f'{sign}{x:.0f}'

In [ ]:
import yfinance as yf
dat = yf.Ticker("MSFT")

In [ ]:
description = dat.info.get('longBusinessSummary')

In [ ]:
dat.get_balance_sheet().index

In [ ]:
#dat.get_info()

In [ ]:
description

In [ ]:
import typing
import fmpsdk
apikey = os.environ.get("FMP")

# Company Valuation Methods
symbol: str = "MSFT"
print(f"Company Profile: {fmpsdk.company_profile(apikey=apikey, symbol=symbol)}")

In [ ]:
msft = fmpsdk.company_profile(apikey=apikey, symbol=symbol)

In [ ]:
msft[0].keys()

In [ ]:
msft[0]['description']

In [ ]:
#fmpsdk.balance_sheet_statement(apikey=apikey, symbol=symbol)

In [ ]:
import pandas as pd

In [ ]:
info = pd.DataFrame(dat.get_info())
type(info)

In [ ]:
#dat.get_news()

In [ ]:
desc = dat.get_info()
type(desc)

In [ ]:
IS = dat.income_stmt

In [ ]:
IS_final = IS.transpose()

In [ ]:
zero_percentage = (IS_final == 0).sum(axis=1) / IS_final.shape[0]
threshold = 0.7
IS_final = IS_final.loc[zero_percentage < threshold]
IS_final

In [ ]:
BS = dat.balance_sheet

In [ ]:
BS_annual = BS.transpose()

In [ ]:
if IS_final.index[-1]=='TTM ':
  BS_quarterly = table_transform(BS[1])
  last_row = BS_quarterly.tail(1)
  BS_annual = pd.concat([BS_annual, last_row])
zero_percentage = (BS_annual == 0).sum(axis=1) / BS_annual.shape[1]
threshold = 0.7
BS_final = BS_annual.loc[zero_percentage < threshold]
BS_final

In [ ]:
CF = dat.cash_flow.iloc[:, :-1]

In [ ]:
CF_final = CF.transpose()
# zero_percentage = (CF_final == 0).sum(axis=1) / CF_final.shape[0]
# threshold = 0.5
# CF_final = CF_final.loc[zero_percentage < threshold]
CF_final

In [ ]:
npm = IS_final['Net Income Common Stockholders']/IS_final['Total Revenue']*100
net_profit = IS_final['Net Income Common Stockholders']
revenue = IS_final['Total Revenue']
ebitda = IS_final['EBITDA']/IS_final['Total Revenue']*100
npm ,ebitda

In [ ]:
#plot 1
categories = IS_final.index
x = np.arange(len(categories))
width = 0.2  # Width of the bars

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Net Profit and Revenue as bar charts
ax1.bar(x - width/2, net_profit, width=width, label='Net Profit', color='royalblue', align='center')
ax1.bar(x + width/2, revenue, width=width, label='Revenue', color='goldenrod', align='center')

# Setting X-axis labels
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.set_ylabel('(Dollar)', fontsize=12)

# Apply the custom formatter to the Y-axis
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))


min_value = min(min(net_profit), min(revenue))
max_value = max(max(net_profit), max(revenue))
ax1.set_ylim([min_value*1.2, max_value*1.2])


# Adding legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
ax1.set_axisbelow(True)

In [ ]:
#plot 2
categories = IS_final.index
x = np.arange(len(categories))

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Net Profit and Revenue as bar charts
ax1.plot(x, npm, color='olivedrab', marker='o', label='Net Profit Margin')
ax1.plot(x, ebitda, color='royalblue', marker='o', label='EBITDA Margin')

#set limit
# min_val = min(min(npm), min(ebitda))
# max_val = max(max(npm), max(ebitda))
# ax1.set_ylim([min_val*1.2, max_val*1.2])

# Setting X-axis labels
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.set_ylabel('(%)', fontsize=12)

# Adding legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

In [ ]:
BS_final.columns

In [ ]:
#Balance sheets
liabilities_long = BS_final['Current Liabilities']
liabilities_short = BS_final['Total Non Current Liabilities Net Minority Interest']
equity = BS_final['Total Equity Gross Minority Interest']
assets = equity + liabilities_long + liabilities_short
non_currect_assets = BS_final['Total Non Current Assets']
DER = (liabilities_long + liabilities_short)/equity*100

#plot 3
categories = BS_final.index
x = np.arange(len(categories))
width = 0.2  # Width of the bars

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Net Profit and Revenue as bar charts
ax1.bar(x - width, equity, width=width, label='Equity', color='royalblue', align='center')
ax1.bar(x, liabilities_long, width=width, label='liabilities(Long Term)', color='red', align='center')
ax1.bar(x, liabilities_short, width=width, label='liabilities(Short Tem)', color='darkred', align='center', bottom=liabilities_long)
ax1.bar(x + width, assets, width=width, label='Assets', color='goldenrod', align='center')

# Setting X-axis labels
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.set_ylabel('(Dollar)', fontsize=12)

# Apply the custom formatter to the Y-axis
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))

# Adding legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
ax1.set_axisbelow(True)

In [ ]:
#plot 4
categories = BS_final.index
x = np.arange(len(categories))

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Net Profit and Revenue as bar charts
ax1.plot(x, DER, color='olivedrab', marker='o', label='Debt to Equity Ratio')

# Setting X-axis labels
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.set_ylabel('(%)', fontsize=12)

# Adding legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
ax1.set_axisbelow(True)

In [ ]:
CF_final.columns

In [ ]:
#Cash Flow
operating = CF_final['Cash Flow From Continuing Operating Activities']
investing = CF_final['Investing Cash Flow']
financing = CF_final['Financing Cash Flow']
beginning_cash = CF_final['Beginning Cash Position']
cash_reserve = beginning_cash + operating + investing + financing
end_cash = CF_final['End Cash Position']

#plot 5
categories = CF_final.index
x = np.arange(len(categories))

# Create figure and axis objects
fig, ax1 = plt.subplots(figsize=(10, 6))

# Plotting Net Profit and Revenue as bar charts
ax1.plot(x, operating, color='olivedrab', marker='o', label='operating cash flow')
ax1.plot(x, investing, color='royalblue', marker='o', label='investing cash flow')
ax1.plot(x, financing, color='darkred', marker='o', label='financing cash flow')

# Setting X-axis labels
ax1.set_xticks(x)
ax1.set_xticklabels(categories)
ax1.set_ylabel('(Dollar)', fontsize=12)
ax1.yaxis.set_major_formatter(ticker.FuncFormatter(format_numbers))

# Adding legends
fig.legend(loc="upper left", bbox_to_anchor=(0.1, 0.9))

ax1.grid(True, axis='y', linestyle='-', linewidth=0.5, color='gray')
ax1.set_axisbelow(True)

In [ ]:
json.dumps(desc)

In [ ]:
#Store in Vector DB
income_statement_json = IS_final.to_json()
cash_flow_json = BS_final.to_json()
balance_sheet_json = CF_final.to_json()
combined_data = {
    "company_description": description,
    "company_all_info": json.dumps(desc),
    "income_statement": json.loads(income_statement_json),
    "cash_flow": json.loads(cash_flow_json),
    "balance_sheet": json.loads(balance_sheet_json)
}

In [ ]:
document = Document(page_content=json.dumps(combined_data))

In [ ]:
import os
ROOT_DIR = os.getcwd()
db_local_folder =  os.path.join(ROOT_DIR, "db3")

In [ ]:
client = QdrantClient(path=db_local_folder)

client.create_collection(
    collection_name="rag-financial",
    vectors_config=VectorParams(size=768, distance=Distance.COSINE),
)

vector_store = QdrantVectorStore(
    client=client,
    collection_name="rag-financial",
    embedding=embeddings,
)


In [ ]:
vector_store.add_documents(documents=[document])

In [ ]:
#retrieval RAG for full analysis
prompt_template = """
        You are a financial analyst. Based on the company profile and the financial reports provided, your task is to provide a detailed financial analysis of the company, focusing on significant increase  or decrease  with a threshold of 10%.
        Use only the financial reports provided in the context below. Do not make assumptions or hallucinate information.

        {context}

        Use the following structure to guide your analysis:
        **Company Profile:**
        - Describe the complete company profile from company_description.

        1. **Income Statement Analysis:**
          - **Revenue Growth:**
            - Detail the number and percentage changes and the periods they occurred. example "2020-2021: Increased by 14.2% (Rp 1,9 to Rp 2,5)"

          - **Gross Profit:**
            - Detail the number and percentage changes and the periods they occurred.

          - **Net Income:**
            - Detail the number and percentage changes and the periods they occurred.

        2. **Cash Flow Statement Analysis:**
          - **Operating Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.

          - **Investing Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Financing Cash Flow:**
            - Detail the number and percentage changes and the periods they occurred.

        3. **Balance Sheet Analysis:**
          - **Total Assets:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Equity:**
            - Detail the number and percentage changes and the periods they occurred.
          - **Debt-to-Equity Ratio:**
            - Detail the number and percentage changes and the periods they occurred.

        4. **Summary:**
          - Summarize the overall financial performance for each period.

        Now, answer the query for this company: {question}
        """

In [ ]:
prompt = ChatPromptTemplate.from_template(prompt_template)

In [ ]:
qa_with_sources = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=vector_store.as_retriever(),
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True,
        )

In [ ]:
analysis = qa_with_sources.invoke("Please provide an overall financial analysis and Information available")
report = f"{analysis['result']}"

In [ ]:
print(report)